# Język Python - Wykład 6

## Docstring

* PEP 257

In [ ]:
class Foo:
    """Represents a Foo"""
    pass

In [ ]:
Foo.__doc__

In [ ]:
def foo_function(arg):
    """Does foo and returns False"""
    return False

In [ ]:
foo_function.__doc__

In [ ]:
def foo_multiline_doc(arg):
    """
    Does foo
    and returns 
    False
    """
    return False

Python *nie* wspiera docstringów dla zmiennych i atrybutów. 

In [ ]:
class A:
    attribute = 3
    """Attribute docstring"""
    
print(A.__doc__)
print(A.attribute.__doc__)

Jednakże niektóre systemy dokumentowania kodu (np. sphinx, epydoc) potrafią parsując kod źródłowy użyć tak skonstruowanych docstringów (warning: it's a hack!)

## "One more thing" (dekoracja)

In [ ]:
def my_decorator(f):
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

In [ ]:
example()

In [ ]:
print(example.__doc__)

In [ ]:
print(example.__name__)

Bazinga!

In [ ]:
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

In [ ]:
print(example.__doc__)
print(example.__name__)

* przy pisaniu dekoratorów **powinniśmy** dekorować funkcję dekorującą dekoratorem wraps z argumentem funkcji dekorowanej
* przypiszemy nowej funkcji wszystkie potrzebne atrybuty starej funkcji, żeby mogła ją całkowicie podmienić 

Atrybuty i metody - mechanizm
http://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/

instance.foobar

![caption](files/L6_img/obj_atr.png)

Class.foobar

![caption](files/L6_img/cl_atr.png)

## Idiomy

### Uruchamianie i import

In [ ]:
def main():
    print('W module: ', __name__)
if __name__ == '__main__':
    main()

### Warunki logiczne

In [ ]:
# GOOD
name = 'Safe'
pets = ['Dog', 'Cat', 'Hamster']
owners = {'Safe': 'Cat', 'George': 'Dog'}
if name and pets and owners:
    print('We have pets!')

In [ ]:
# NOT SO GOOD
if name != '' and len(pets) > 0 and owners != {}:
    print('We have pets!')

Przynależność

In [ ]:
# GOOD
name = 'Safe Hammad'
if 'H' in name:
    print('This name has an H in it!')

In [ ]:
# NOT SO GOOD
name = 'Safe Hammad'
if name.find('H') != -1:
    print('This name has an H in it!')

In [ ]:
# GOOD
pets = ['Dog', 'Cat', 'Hamster']
ages = [5,2,3]
for pet,age in zip(pets,ages):
    print(pet, 'is', age, 'years old')

In [ ]:
# NOT SO GOOD
pets = ['Dog', 'Cat', 'Hamster']
ages = [5,2,3]
i = 0
while i < len(pets):
    print(pets[i], 'is', ages[i], 'years old')
    i += 1

### Składanie tekstu

In [ ]:
# GOOD
chars = ['S', 'a', 'f', 'e']
name = ''.join(chars)
print(name)

In [ ]:
# NOT SO GOOD
chars = ['S', 'a', 'f', 'e']
name = ''
for char in chars:
    name += char
print(name)

### EAFP vs LBYL
“It's Easier to Ask for Forgiveness than
Permission.”
“Look Before You Leap”

In [ ]:
# GOOD
d = {'x': '5'}
try:
    value = int(d['x'])
except (KeyError, TypeError, ValueError):
    value = None

In [ ]:
# NOT SO GOOD
d = {'x': '5'}
if 'x' in d and isinstance(d['x'], str) and d['x'].isdigit():
    value = int(d['x'])
else:
    value = None

### List comprehension

In [ ]:
# GOOD
data = [7, 20, 3, 15, 11]
result = [i * 3 for i in data if i > 10]
print(result)

In [ ]:
# NOT SO GOOD (MOST OF THE TIME)
data = [7, 20, 3, 15, 11]
result = []
for i in data:
    if i > 10:
        result.append(i * 3)
print(result)

### Upraszczanie wyboru

In [ ]:
number_type = '?'
if number_type == 'floating_point':
    converter = float
elif number_type == 'integer':
    converter = int
else:
    converter = complex
converter("12")

In [ ]:
number_type = '?'
converters = {
    'floating_point' : float,
    'integer' : int,
}
converter = converters.get( number_type , complex )
converter("12")

## Wzorce projektowe

### Wbudowane:
  - iterator
  - dekorator

### Zasoby:
    https://github.com/faif/python-patterns

### Singleton

Definicja:
 -  jeden obiekt danej klasy

Zastosowania:
 * potrzeba tylko jednego obiektu (np. połączenie do bazy danych, logger)
 * kontrola współbieżnego dostępu do współdzielonego zasobu
 * singleton tylko-do-odczytu dla globalnego stanu (np. strefa czasowa, język, stałe fizyczne) 

In [ ]:
class Singleton():
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super(Singleton, cls).__new__(cls)
        return cls.instance

In [ ]:
o1 = Singleton()
o2 = Singleton()

In [ ]:
o1.x = 2

In [ ]:
print(o2.x)

In [ ]:
id(o1),id(o2)

In [ ]:
o1 is o2, o1 == o2

### Dzielenie stanu

In [ ]:
class Borg():
    _shared_state = {}
    def __new__(cls, *args, **kwargs):
        obj = super(Borg, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

In [ ]:
b1 = Borg()
b2 = Borg()

In [ ]:
b1.x = 3
b2.x

In [ ]:
id(b1),id(b2)

In [ ]:
b1 is b2

https://chromium.googlesource.com/external/googleappengine/python/+/master/google/pyglib/singleton.py

## Pułapki importowania

http://ncoghlan-devs-python-notes.readthedocs.org/en/latest/python_concepts/import_traps.html

project/
    setup.py
    example/
        __init__.py
        foo.py
        tests/
            __init__.py
            test_foo.py

// These commands will most likely *FAIL* due to problems with the way
//the import state gets initialised, even if the test code is correct

// working directory: project/example/tests
./test_foo.py
python test_foo.py
python -m test_foo
python -c "from test_foo import main; main()"

// working directory: project/example
tests/test_foo.py
python tests/test_foo.py
python -m tests.test_foo
python -c "from tests.test_foo import main; main()"

// working directory: project
example/tests/test_foo.py
python example/tests/test_foo.py

// working directory: project/..
project/example/tests/test_foo.py
python project/example/tests/test_foo.py
python -m project.example.tests.test_foo
python -c "from project.example.tests.test_foo import main; main()"

In [ ]:
# working directory: project
python -c "from example.tests.test_foo import main; main()"

In [ ]:
# working directory: project
python -m example.tests.test_foo

In [ ]:
https://mail.python.org/pipermail/python-3000/2007-April/006793.html

### MVC

Definicja:
 - model  - DANE i LOGIKA BIZNESOWA (nie zależy od widoku i kontrolera)
 - widok (view) - PREZENTACJA (zależy od modelu)
 - kontroler (controller) - ŁĄCZY WIDOK i MODEL (zależy od modelu i widoku)

In [ ]:
class View():

    def product_list(self, product_list):
        print('PRODUCT LIST:')
        for product in product_list:
            print(product)
        print('')

    def product_information(self, product, product_info):
        print('PRODUCT INFORMATION:')
        print('Name: %s, Price: %.2f, Quantity: %d\n' %
              (product.title(), product_info.get('price', 0),
               product_info.get('quantity', 0)))

    def product_not_found(self, product):
        print('That product "%s" does not exist in the records' % product)

In [ ]:
class Model():

    products = {
        'milk': {'price': 1.50, 'quantity': 10},
        'eggs': {'price': 0.20, 'quantity': 100},
        'cheese': {'price': 2.00, 'quantity': 10}
    }

In [ ]:
class Controller():

    def __init__(self, model, view):
        self.model = model
        self.view = view

    def get_product_list(self):
        product_list = self.model.products.keys()
        self.view.product_list(product_list)

    def get_product_information(self, product):
        product_info = self.model.products.get(product, None)
        if product_info is not None:
            self.view.product_information(product, product_info)
        else:
            self.view.product_not_found(product)

In [ ]:
controller = Controller(Model(),View())
controller.get_product_list()
controller.get_product_information('cheese')
controller.get_product_information('eggs')
controller.get_product_information('milk')
controller.get_product_information('arepas')

In [ ]:
class ViewOther():

    def product_list(self, product_list):
        print('PRODUCT LIST:')
        for product in product_list:
            print(" - ", product)
        print('')

In [ ]:
controller = Controller(Model(),ViewOther())
controller.get_product_list()

## Factory method pattern

In [ ]:
class A():
    def __init__(self):
        self.a = "Hello"

class B():
    def __init__(self):
        self.a = " World"
    
myfactory = {
    "greeting" : A,
    "subject" : B,
}

In [ ]:
myfactory["greeting"]().a

## Strategia

In [ ]:
def bisection(line):
    return 5.5, 6.6
def conjugate_gradient(line):
    return 3.3, 4.4
def test():
    solver = conjugate_gradient
    print(solver((5.5,5.5)))
    solver = bisection
    print(solver((5.5,5.5)))
test() 

## Observer

In [ ]:
class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def scale(self, n):
        self.x = n * self.x
        self.y = n * self.y
        
def notify(f):
    def g(self, n):
        print("executed", n)
        return f(self, n)
    return g

Point.scale = notify(Point.scale)
p = Point(2.0, 3.0)
p.scale(2.5)

In [ ]:
def notify(f):
    def g(self, n):
        print("executed", n)
        return f(self, n)
    return g

class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    @notify
    def scale(self, n):
        self.x = n * self.x
        self.y = n * self.y
        

p = Point(2.0, 3.0)
p.scale(2.5)